In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix, roc_curve, auc
import torch.optim as optim
import torch.nn.functional as F
import time

In [ ]:
def result_graph():
    plt.figure(1)
    
    plt.subplot(2,1,1)
    plt.plot(range(1,len(train_accuracy)+1),train_accuracy,'b',label = 'train accuracy')
    plt.plot(range(1,len(val_accuracy)+1),val_accuracy,'r',label = 'valid accuracy')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.title('Acc Curve')
    plt.legend()
    plt.subplots_adjust(hspace=0.7)
    
    plt.subplot(2,1,2)
    plt.plot(range(1,len(train_losses)+1),train_losses,'b',label = 'train loss')
    plt.plot(range(1,len(val_losses)+1),val_losses,'r',label = 'valid loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss Curve')
    plt.legend()
    

# Training loop function

In [ ]:
def fit(epoch,model,data_loader,phase='train',volatile=False):

    if phase == 'train':
        model.train()
    if phase == 'valid':
        model.eval()

    running_loss = 0.0
    running_correct = 0

    if phase == 'valid':
        with torch.no_grad():
            for batch_idx , (inputs,target) in enumerate(data_loader):
                inputs,target = inputs.cuda(),target.cuda()
                outputs = model(inputs)
                loss = criterion(outputs,target)  
                running_loss += loss.data.item()
                _, preds = torch.max(outputs, 1)
                running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
            loss = running_loss/len(data_loader.dataset)
            accuracy = 100. * running_correct/len(data_loader.dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, loss, accuracy))
            
    if phase == 'train':
        with torch.set_grad_enabled(phase == 'train'):
            for batch_idx , (inpust,target) in enumerate(data_loader):
                inputs,target = inpust.cuda(),target.cuda()
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output,target)  
                running_loss += loss.data.item()
                _, preds = torch.max(output, 1)
                loss.backward()
                optimizer.step()
                running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        loss = running_loss/len(data_loader.dataset)
        accuracy = 100. * running_correct/len(data_loader.dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, loss, accuracy))

    
   
    return loss,accuracy

# Evaluation function

In [ ]:
def inference(loader):
    
    model.eval()
    running_correct = 0
    nb_classes = 2
    roc_max_diff, roc_target = [],[]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for inputs,target in loader:
            inputs,target = inputs.cuda(),target.cuda()
            outputs_original = model(inputs)
            outputs_roc = F.softmax(outputs_original,dim = 1)
            outputs_roc = outputs_roc.cpu()
            outputs_roc = outputs_roc.numpy()

            for i in range(len(target)):
                roc_max_diff = np.append(roc_max_diff,outputs_roc[i,1])

            load_original_preds = outputs_original.cpu()
            load_original_preds = load_original_preds.numpy()
            _, preds = torch.max(outputs_original, 1)
            preds,target,inputs = preds.cpu(),target.cpu(),inputs.cpu()
            running_correct += preds.eq(target.data.view_as(preds)).cpu().sum() 
            roc_target = np.append(roc_target,target)

            for t, p in zip(target.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        accuracy = 100. * running_correct/len(loader.dataset)
                    
    s = [['TN','FP'], ['FN', 'TP']]
    for i in range(2):
        for j in range(2):
            s[i][j] = confusion_matrix[i][j]
    TN,FP,FN,TP = s[0][0],s[0][1],s[1][0],s[1][1]
    PE = ((TP+FN)/(len(loader.dataset)))*((TP+FP)/(len(loader.dataset)))+((FP+TN)/(len(loader.dataset)))*((FN+TN)/(len(loader.dataset)))
    print(confusion_matrix)
    print(confusion_matrix.diag()/confusion_matrix.sum(1))
    print('TP = {}, FP = {}, TN = {}, FN = {}'.format(TP,FP,TN,FN))
    print('Specifity = {:.6f}, Sensitivity = {:.6f}'.format(TN/(TN+FP),TP/(TP+FN)))
    print('F1 score = {:.6f}'.format(TP/(TP+(FN+FP)/2)))

    print('Test Acc = {:.6f}'.format(accuracy))
    accuracy = accuracy.type(torch.FloatTensor)
    PE = PE.type(torch.FloatTensor)
    Kappa = (0.01*accuracy-PE)/(1.0-PE)
    Kappa = Kappa.type(torch.FloatTensor)
    print('cohens kappa = {:.6f}'.format(Kappa))
    fpr, tpr, _ = roc_curve(roc_target,roc_max_diff)
    roc_auc = auc(fpr,tpr)
    print('AUC = {:.6f}'.format(roc_auc))

# Model and parameter setting

In [ ]:
batch_size = 16
num_epochs = 40
transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

model = models.resnet50(pretrained=True)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 2)
    
model.dropout = nn.Dropout(p=0.5)

if torch.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.000001)    

# Train and validation dataset setting

In [ ]:
train_dataset_folder_path = 
validation_datset_folder_path = 

train = ImageFolder(train_dataset_folder_path,transform=transform)
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

valid = ImageFolder(validation_datset_folder_path,transform=transform)
validloader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# Training loop

In [ ]:
since = time.time()
train_losses, train_accuracy = [],[]
val_losses, val_accuracy = [],[]

for epoch in range(num_epochs):
    print()
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    epoch_loss, epoch_accuracy = fit(epoch,model,trainloader,phase='train')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,validloader,phase='valid')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    
result_graph()
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
time_elapsed // 60, time_elapsed % 60))

# Evaluation of trained model

In [ ]:
test_dataset_folder_path = 
test = ImageFolder(test_dataset_folder_path,transform=transform)
testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
inference(testloader)